# STEP 2: Autoencoder per Anomaly Detection

Autoencoder convoluzionale addestrato solo su immagini OK (visibili e corrette).
Calcola threshold basato su errore di ricostruzione per rilevare anomalie.


In [ ]:
# Setup: Clona repository GitHub e monta Google Drive per i dati
import os
from pathlib import Path

# Opzione 1: Clona da GitHub (consigliato per sviluppo)
# Sostituisci con il tuo repository URL
GITHUB_REPO = "https://github.com/Giovanni000/Project-Work.git"  # ⚠️ MODIFICA QUESTO!
REPO_DIR = "/content/project"

# Clona repository (se non esiste già)
if not Path(REPO_DIR).exists():
    !git clone {GITHUB_REPO} {REPO_DIR}

# Cambia directory al repository
os.chdir(REPO_DIR)
print(f"Repository directory: {os.getcwd()}")

# Opzione 2: Monta Google Drive solo per i dati (immagini)
from google.colab import drive
drive.mount('/content/drive')

# Path ai dati su Drive
DATA_ROOT = Path("/content/drive/MyDrive/Project Work/Data")
print(f"Data directory: {DATA_ROOT}")

# Import necessari
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
from pathlib import Path
import numpy as np
from tqdm import tqdm

# Seed per riproducibilità
torch.manual_seed(42)
np.random.seed(42)

# Verifica device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")


## Dataset Class (solo OK)


In [ ]:
class AEDataset(Dataset):
    """
    Dataset PyTorch per autoencoder.
    Contiene solo immagini con label == "OK".
    """
    
    def __init__(self, csv_path, transform=None):
        """
        Args:
            csv_path: Path al CSV con colonne 'image_path' e 'label'
            transform: Trasformazioni da applicare alle immagini
        """
        df = pd.read_csv(csv_path)
        # Filtra solo OK
        self.df = df[df['label'] == 'OK'].copy().reset_index(drop=True)
        self.transform = transform
        
        print(f"Dataset AE caricato: {len(self.df)} immagini OK")
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = row['image_path']
        
        # Carica immagine
        image = Image.open(image_path).convert('RGB')
        
        # Applica trasformazioni
        if self.transform:
            image = self.transform(image)
        
        return image


## Modello Autoencoder Convoluzionale


In [ ]:
class ConvAE(nn.Module):
    """
    Autoencoder convoluzionale per anomaly detection.
    
    Encoder: 3 conv2d con stride=2 (3→16→32→64 canali)
    Decoder: 3 convtranspose2d simmetriche (64→32→16→3 canali)
    """
    
    def __init__(self):
        super(ConvAE, self).__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            # 128x128x3 -> 64x64x16
            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            
            # 64x64x16 -> 32x32x32
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            
            # 32x32x32 -> 16x16x64
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            # 16x16x64 -> 32x32x32
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            
            # 32x32x32 -> 64x64x16
            nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            
            # 64x64x16 -> 128x128x3
            nn.ConvTranspose2d(16, 3, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()  # Output in [0, 1]
        )
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


## Training Loop


In [ ]:
def train_autoencoder(csv_path="data/dataset.csv",
                     batch_size=32,
                     num_epochs=30,
                     learning_rate=0.001,
                     device=None):
    """
    Training loop per l'autoencoder.
    """
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    print(f"Device: {device}")
    
    # Trasformazioni per le immagini (normalizzazione in [0,1])
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),  # Già in [0, 1]
    ])
    
    # Crea dataset (solo OK)
    dataset = AEDataset(csv_path, transform=transform)
    
    # DataLoader
    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    
    print(f"\nDataset: {len(dataset)} immagini OK")
    print(f"Batch size: {batch_size}")
    print(f"Numero di batch: {len(train_loader)}")
    
    # Modello
    model = ConvAE().to(device)
    print(f"\nModello creato. Parametri totali: {sum(p.numel() for p in model.parameters()):,}")
    
    # Loss e optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Training loop
    print(f"\nInizio training per {num_epochs} epoch...")
    print("-" * 60)
    
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0
        
        for images in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            images = images.to(device)
            
            optimizer.zero_grad()
            reconstructed = model(images)
            loss = criterion(reconstructed, images)
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
        
        avg_loss = epoch_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{num_epochs}: Reconstruction Loss = {avg_loss:.6f}")
        print("-" * 60)
    
    print(f"\n✅ Training completato!")
    
    # Salva modello
    models_dir = Path("models")
    models_dir.mkdir(exist_ok=True)
    model_path = models_dir / "ae_conv.pth"
    torch.save(model.state_dict(), model_path)
    print(f"   Modello salvato in: {model_path}")
    
    return model


## Esegui Training


In [ ]:
# Training dell'autoencoder
model_ae = train_autoencoder(
    csv_path="data/dataset.csv",
    batch_size=32,
    num_epochs=30,
    learning_rate=0.001,
    device=device
)


## Calcolo Threshold

Calcola il threshold per anomaly detection basato su errore di ricostruzione.
Threshold = mu + 3*sigma


In [ ]:
def calculate_threshold(model, csv_path="data/dataset.csv", device=None):
    """
    Calcola il threshold per anomaly detection basato su errore di ricostruzione.
    
    Threshold = mu + 3*sigma, dove mu e sigma sono media e std degli errori
    di ricostruzione su tutto il dataset di training (solo OK).
    """
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    print("\nCalcolo threshold su dataset di training...")
    
    # Trasformazioni
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
    ])
    
    # Dataset (solo OK)
    dataset = AEDataset(csv_path, transform=transform)
    loader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=2)
    
    model.eval()
    errors = []
    criterion = nn.MSELoss(reduction='none')  # Per avere errore per ogni pixel
    
    with torch.no_grad():
        for images in tqdm(loader, desc="Calcolo errori"):
            images = images.to(device)
            reconstructed = model(images)
            
            # Errore per ogni immagine nel batch
            # Shape: [batch, 3, 128, 128]
            batch_errors = criterion(reconstructed, images)
            # Media su canali e spaziale: [batch]
            batch_errors = batch_errors.mean(dim=(1, 2, 3))
            
            errors.extend(batch_errors.cpu().numpy())
    
    errors = np.array(errors)
    mu = np.mean(errors)
    sigma = np.std(errors)
    threshold = mu + 3 * sigma
    
    print(f"\nStatistiche errori di ricostruzione:")
    print(f"  Media (mu): {mu:.6f}")
    print(f"  Std (sigma): {sigma:.6f}")
    print(f"  Min: {errors.min():.6f}")
    print(f"  Max: {errors.max():.6f}")
    print(f"\nThreshold (mu + 3*sigma): {threshold:.6f}")
    
    # Salva threshold
    models_dir = Path("models")
    models_dir.mkdir(exist_ok=True)
    threshold_path = models_dir / "ae_threshold.npy"
    np.save(threshold_path, threshold)
    print(f"  Threshold salvato in: {threshold_path}")
    
    return threshold


In [ ]:
# Calcola il threshold
threshold = calculate_threshold(model_ae, csv_path="data/dataset.csv", device=device)
print(f"\n✅ Threshold calcolato: {threshold:.6f}")


## Funzione Helper per Caricare Modello e Threshold


In [ ]:
def load_ae_and_threshold(device=None):
    """
    Carica l'autoencoder addestrato e il threshold.
    """
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Carica modello
    model = ConvAE().to(device)
    model_path = Path("models/ae_conv.pth")
    
    if not model_path.exists():
        raise FileNotFoundError(f"Modello non trovato: {model_path}")
    
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    
    # Carica threshold
    threshold_path = Path("models/ae_threshold.npy")
    if not threshold_path.exists():
        raise FileNotFoundError(f"Threshold non trovato: {threshold_path}")
    
    threshold = np.load(threshold_path)
    
    return model, threshold
